## LBP para geração de histograma

In [20]:
'''
Método de extração de arquivo .arff. Autor: Glauco Vitor.
'''

def gravar_arquivo_arff(base_teste, classes): # 0 para NAO_MELANOMA e 1 para MELANOMA
    tam = len(base_teste[0][0])
    file = open('lbp.arff','w') 
 
    file.write('@relation cancer\n') 
    for i in range(tam):
        file.write('@attribute '+ str(i) +' NUMERIC\n') 
    
    file.write('@attribute classes {')
    
    a = set(classes)
    
    for i in a:
        file.write(str(i)+',')
    
    file.write('}')    
    
    for i in range(tam):
         len(set(classes))
    
    file.write('\n@data\n') 

    for item in base_teste:
        for i in range(len(item[0])):
            file.write("%s," % str(item[0][i])) 
        file.write("%s\n" % item[1])    
 
    file.close() 
    print('arquivo gravado')


In [21]:
'''
Distância euclidiana para a atribuição dos centróides a cada elemento da imagem.
'''

def distancia(a, b):
    M = len(a)
    soma = 0
    for i in range(M):
        soma = soma + ((a[i]-b[i])**2)
    return np.sqrt(soma)


In [22]:
from skimage import feature
import numpy as np
 


def extrair_caracteristica(image):
    # compute the Local Binary Pattern representation
    # of the image, and then use the LBP representation
    # to build the histogram of patterns
    
    numPoints = 8
    radius = 1
    eps=1e-7
    
    lbp = feature.local_binary_pattern(image, numPoints, radius, method="default")
    (hist, _) = np.histogram(lbp.ravel(),
        bins = np.arange(0, 257),
        range = (0, numPoints + 2))

    # normalize the histogram
    hist = hist.astype("float")
    hist /= (hist.sum() + eps)
    
    # return the histogram of Local Binary Patterns
    return hist  # hist.lenght = K + 2

In [23]:
import cv2 as cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.cluster import KMeans
from sklearn import metrics

files_path_nao_melanoma = os.listdir(PATH_NOTMELANOMA) 
files_path_nao_melanoma = [PATH_NOTMELANOMA+'/'+path for path in files_path_nao_melanoma]
files_path_melanoma = os.listdir(PATH_MELANOMA)
files_path_melanoma = [PATH_MELANOMA+'/'+path for path in files_path_melanoma]


all_image_paths = [files_path_nao_melanoma, files_path_melanoma]

base = []
classes = []

#extraindo as características das imagens na base de dados
for i in range(len(all_image_paths)):
    for img_path in all_image_paths[i][:500]:
        imagem = cv2.imread(img_path)
        imagem = cv2.cvtColor(imagem, cv2.COLOR_BGR2GRAY)
        caracteristica = extrair_caracteristica(imagem)
        classes.append(i)
        base.append((caracteristica, i))
    print('Classe final: '+str(i))
    print('Size hist: ')
    print(len(caracteristica))

gravar_arquivo_arff(base, classes)

Classe final: 0
Size hist: 
256
Classe final: 1
Size hist: 
256
arquivo gravado
